In [ ]:
# Adrian Marinovich
# Springboard - Data Science Career Track
# SQL Exercise
# September 5, 2018

In [122]:
# Create local Country Club database (using SQLite):
#  Obtain tables as csv flat files 
#    (first must remove ',' in Members addresses).
#  In termimal:
#    $ sqlite3 countryclub.db
#    sqlite> .mode csv
#    sqlite> .separator ","
#    sqlite> .import /<path>/Bookings.csv bookings
#    sqlite> .schema bookings
#    CREATE TABLE bookings(
#      "bookid" TEXT,
#      "facid" TEXT,
#      "memid" TEXT,
#      "starttime" TEXT,
#      "slots" TEXT
#    );
#    sqlite> .import /home/adrian/awork/ds/springboard_exercises/DSCT/data_wrangling_sql/Facilities.csv facilities
#    sqlite> .schema facilities
#    CREATE TABLE facilities(
#      "facid" TEXT,
#      "name" TEXT,
#      "membercost" TEXT,
#      "guestcost" TEXT,
#      "initialoutlay" TEXT,
#      "monthlymaintenance" TEXT
#    );
#    sqlite> .import /home/adrian/awork/ds/springboard_exercises/DSCT/data_wrangling_sql/Members.csv members
#    sqlite> .schema members
#    CREATE TABLE members(
#      "memid" TEXT,
#      "surname" TEXT,
#      "firstname" TEXT,
#      "address" TEXT,
#      "zipcode" TEXT,
#      "telephone" TEXT,
#      "recommendedby" TEXT,
#      "joindate" TEXT
#    );
#    

import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'data/countryclub.db')

# If necessary, create dataframes from queries
#members = pd.read_sql_query("SELECT * FROM members", cnx)
#facilities = pd.read_sql_query("SELECT * FROM facilities", cnx)
#bookings = pd.read_sql_query("SELECT * FROM bookings", cnx)

In [123]:
# Q1: Some of the facilities charge a fee to members, but some do not. 
#     Please list the names of the facilities that do.

pd.read_sql_query('''SELECT name, membercost 
                     FROM facilities 
                    WHERE CAST(membercost as REAL) > 0.0''', cnx)

,name,membercost
0,Tennis Court 1,5.0
1,Tennis Court 2,5.0
2,Massage Room 1,9.9
3,Massage Room 2,9.9
4,Squash Court,3.5


In [124]:
# Q2: How many facilities do not charge a fee to members?

pd.read_sql_query('''SELECT COUNT(name) FROM facilities 
                     WHERE CAST(membercost as REAL) = 0.0''', cnx)

,COUNT(name)
0,4


In [125]:
# Q3: How can you produce a list of facilities that charge a fee 
#   to members, where the fee is less than 20% of the facility's 
#   monthly maintenance cost? Return the facid, facility name, 
#   member cost, and monthly maintenance of the facilities in question. 

pd.read_sql_query('''SELECT facid, name, membercost, monthlymaintenance 
                     FROM facilities 
                     WHERE CAST(membercost as REAL)
                         AND (CAST(membercost as REAL)/CAST(monthlymaintenance as REAL)) < 0.2''', cnx)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


In [126]:
# Q4: How can you retrieve the details of facilities with ID 1 and 5? 
#   Write the query without using the OR operator.

pd.read_sql_query('''SELECT *
                     FROM facilities 
                     WHERE facid IN (1, 5)''', cnx)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


In [127]:
# Q5: How can you produce a list of facilities, with each labelled 
#   as 'cheap' or 'expensive', depending on if their monthly 
#   maintenance cost is more than $100? Return the name and monthly 
#   maintenance of the facilities in question. 

pd.read_sql_query('''SELECT name, monthlymaintenance,
                     CASE WHEN CAST(monthlymaintenance as REAL) < 100 
                         THEN "cheap"
                         ELSE "expensive"
                         END AS cheap_expensive 
                     FROM facilities''', cnx)

,name,monthlymaintenance,cheap_expensive
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


In [128]:
# Q6: You'd like to get the first and last name of the last 
#   member(s) who signed up. 
# Do not use the LIMIT clause for your solution.

pd.read_sql_query('''SELECT firstname, surname
                     FROM members 
                     WHERE joindate=(SELECT MAX(joindate) FROM members)''', cnx)

,firstname,surname
0,Darren,Smith


In [129]:
# Q7: How can you produce a list of all members who have used 
#   a tennis court? 
# Include in your output the name of the court, 
#   and the name of the member formatted as a single column. 
# Ensure no duplicate data, and order by the member name. 

pd.read_sql_query('''SELECT facilities.name AS Facility_name, 
                        members.firstname||" "||members.surname AS Member_name
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     JOIN members 
                         ON bookings.memid = members.memid 
                     WHERE facilities.facid IN (0, 1) 
                     GROUP BY Member_Name
                     ORDER BY Member_Name''', cnx)

,Facility_name,Member_name
0,Tennis Court 2,Anne Baker
1,Tennis Court 2,Burton Tracy
2,Tennis Court 2,Charles Owen
3,Tennis Court 2,Darren Smith
4,Tennis Court 2,David Farrell
5,Tennis Court 2,David Jones
6,Tennis Court 1,David Pinker
7,Tennis Court 1,Douglas Jones
8,Tennis Court 1,Erica Crumpet
9,Tennis Court 2,Florence Bader


In [130]:
# Q8: How can you produce a list of bookings on the day 
#   of 2012-09-14 which will cost the member (or guest) 
#   more than $30? 
# Remember that guests have different costs to members 
#   (the listed costs are per half-hour 'slot'), and the 
#   guest user's ID is always 0. 
# Include in your output the name of the facility, the 
#   name of the member formatted as a single column, and 
#   the cost. 
# Order by descending cost, and do not use 
#   any subqueries. 

pd.read_sql_query('''SELECT facilities.name AS Facility_name,
                        members.firstname||" "||members.surname AS Member_name,
                        CAST(facilities.membercost as REAL)*CAST(bookings.slots as REAL) AS Cost
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     JOIN members 
                         ON bookings.memid = members.memid 
                     WHERE SUBSTR(bookings.starttime, 1, 10) = "2012-09-14"
                         AND Cost > 30
                         AND members.memid != 0
                     
                     UNION

                     SELECT facilities.name AS Facility_name, 
                        members.firstname||" "||members.surname AS Member_name,
                        CAST(facilities.guestcost as REAL)*CAST(bookings.slots as REAL) AS Cost
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     JOIN members 
                         ON bookings.memid = members.memid 
                     WHERE SUBSTR(bookings.starttime, 1, 10) = "2012-09-14"
                         AND Cost > 30
                         AND members.memid = 0 

                     ORDER BY Cost DESC ''', cnx)

,Facility_name,Member_name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Tennis Court 2,GUEST GUEST,150.0
3,Tennis Court 1,GUEST GUEST,75.0
4,Tennis Court 2,GUEST GUEST,75.0
5,Squash Court,GUEST GUEST,70.0
6,Massage Room 1,Jemima Farrell,39.6
7,Squash Court,GUEST GUEST,35.0


In [131]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

pd.read_sql_query('''SELECT sub.* 
                     FROM (

                     SELECT facilities.name AS Facility_name,
                        members.firstname||" "||members.surname AS Member_name,
                        CAST(facilities.membercost as REAL)*CAST(bookings.slots as REAL) AS Cost
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     JOIN members 
                         ON bookings.memid = members.memid 
                     WHERE SUBSTR(bookings.starttime, 1, 10) = "2012-09-14"
                         AND Cost > 30
                         AND members.memid != 0
                     
                     UNION

                     SELECT facilities.name AS Facility_name, 
                        members.firstname||" "||members.surname AS Member_name,
                        CAST(facilities.guestcost as REAL)*CAST(bookings.slots as REAL) AS Cost
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     JOIN members 
                         ON bookings.memid = members.memid 
                     WHERE SUBSTR(bookings.starttime, 1, 10) = "2012-09-14"
                         AND Cost > 30
                         AND members.memid = 0 

                     ) sub
                     
                     ORDER BY Cost DESC''', cnx)

,Facility_name,Member_name,Cost
0,Massage Room 2,GUEST GUEST,320.0
1,Massage Room 1,GUEST GUEST,160.0
2,Tennis Court 2,GUEST GUEST,150.0
3,Tennis Court 1,GUEST GUEST,75.0
4,Tennis Court 2,GUEST GUEST,75.0
5,Squash Court,GUEST GUEST,70.0
6,Massage Room 1,Jemima Farrell,39.6
7,Squash Court,GUEST GUEST,35.0


In [132]:
# Q10: Produce a list of facilities with a total revenue 
#   less than 1000. 
# The output of facility name and total revenue, sorted by revenue. 
# Remember that there's a different cost for guests and members! 

pd.read_sql_query('''SELECT facilities.name AS Facility_name, 
                            SUM(
                                CASE 
                                  WHEN bookings.memid != 0
                                    THEN CAST(facilities.membercost as REAL)*CAST(bookings.slots as REAL) 
                                  WHEN bookings.memid = 0
                                    THEN CAST(facilities.guestcost as REAL)*CAST(bookings.slots as REAL) 
                                  ELSE 0
                                END  
                               ) AS Revenue 
                     FROM facilities  
                     JOIN bookings
                         ON facilities.facid = bookings.facid
                     GROUP BY Facility_name
                     HAVING Revenue <1000
                     ORDER BY Revenue DESC''', cnx)

,Facility_name,Revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0


In [25]:
cnx.close()